In [1]:
import sys

sys.path.append('../')

In [2]:
import logging
import time

import jax.numpy as jnp
import pandas as pd

from src.algorithms.parallel_jax_pso import parallel_jax_pso
from src.config import config

In [3]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)
logger = logging.getLogger(__name__)

In [4]:
def benchmark_algorithm(
    algorithm_name: str,
    algorithm_fn: callable,
    objective_fn: callable,
    bounds: tuple,
    params: dict,
    num_runs: int,
    num_subswarms: int,
) -> float:
    execution_times = []

    for i in range(num_runs):
        logger.info(
            '%s | Run %d/%d | [RUNNING]',
            algorithm_name,
            i + 1,
            num_runs,
        )

        start_time = time.perf_counter()

        if algorithm_name == 'JAX PSO':
            parallel_jax_pso(objective_fn, bounds, params, num_subswarms)
        else:
            algorithm_fn(objective_fn, bounds, **params)

        elapsed = time.perf_counter() - start_time
        execution_times.append(elapsed)

        logger.info(
            '%s | Run %d/%d | [DONE] in %.6fs',
            algorithm_name,
            i + 1,
            num_runs,
            elapsed,
        )

    return execution_times

def compare_pso_implementations(
    benchmarks: dict,
    algorithms: dict,
    dims: list,
    num_runs: int,
    num_subswarms: int,
    params: dict,
) -> dict:

    logger.info('='*60)
    logger.info(
        'INITIATING BENCHMARKS | Runs: %d | Subswarms: %d',
        num_runs,
        num_subswarms,
    )
    logger.info('='*60)

    results = {
        dim: {
            benchmark_name: {
                algorithm_name: [] for algorithm_name in algorithms
            } for benchmark_name in benchmarks
        } for dim in dims
    }

    for dim in dims:
        params['num_dims'] = dim

        for benchmark_name, (objective_fn, bounds) in benchmarks.items():
            for name, algorithm in algorithms.items():
                logger.info(
                    '>>> Dimension: %d | Benchmark: %s | Algorithm: %s',
                    dim,
                    benchmark_name,
                    name,
                )

                results[dim][benchmark_name][name] = benchmark_algorithm(
                    name,
                    algorithm,
                    objective_fn,
                    bounds,
                    params,
                    num_runs,
                    num_subswarms,
                )

    logger.info('='*60)
    logger.info('COMPLETED | All benchmarks finished successfully')
    logger.info('='*60)
    return results

def save_results(results: dict) -> None:
    rows = []

    for dim, benchmarks_data in results.items():
        for benchmark_name, algorithms_data in benchmarks_data.items():
            for algorithm_name, execution_times in algorithms_data.items():
                mean_time = float(jnp.mean(jnp.array(execution_times)))

                rows.extend(
                    [
                        {
                            'Dimension': dim,
                            'Benchmark': benchmark_name,
                            'Algorithm': algorithm_name,
                            'Execution Times': execution_times,
                            'Mean of Execution Times (s)': mean_time,
                        },
                    ],
                )

    df = pd.DataFrame(rows)
    df.to_csv('../data/results.csv')

In [5]:
results = compare_pso_implementations(
    config['benchmarks'],
    config['algorithms'],
    config['dims'],
    config['num_runs'],
    config['num_subswarms'],
    config['params'],
)

2026-01-06 08:17:39 [INFO] ============================================================
2026-01-06 08:17:39 [INFO] INITIATING BENCHMARKS | Runs: 10 | Subswarms: 10
2026-01-06 08:17:39 [INFO] ============================================================
2026-01-06 08:17:39 [INFO] >>> Dimension: 2 | Benchmark: Ackley | Algorithm: Pure PSO
2026-01-06 08:17:39 [INFO] Pure PSO | Run 1/10 | [RUNNING]
2026-01-06 08:17:39 [INFO] Pure PSO | Run 1/10 | [DONE] in 0.137240s
2026-01-06 08:17:39 [INFO] Pure PSO | Run 2/10 | [RUNNING]
2026-01-06 08:17:39 [INFO] Pure PSO | Run 2/10 | [DONE] in 0.132900s
2026-01-06 08:17:39 [INFO] Pure PSO | Run 3/10 | [RUNNING]
2026-01-06 08:17:39 [INFO] Pure PSO | Run 3/10 | [DONE] in 0.124425s
2026-01-06 08:17:39 [INFO] Pure PSO | Run 4/10 | [RUNNING]
2026-01-06 08:17:39 [INFO] Pure PSO | Run 4/10 | [DONE] in 0.121805s
2026-01-06 08:17:39 [INFO] Pure PSO | Run 5/10 | [RUNNING]
2026-01-06 08:17:40 [INFO] Pure PSO | Run 5/10 | [DONE] in 0.122829s
2026-01-06 08:17:40 [I

In [6]:
save_results(results)